In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from gensim.models import word2vec
import nltk.data
import re
import logging
import pickle as pkl
from scipy import spatial

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
with open("/kaggle/input/skills/prep_data_tokens_underscore_1.txt", "rb") as g:
    data_dice = pkl.load(g)

print(len(data_dice))
print(data_dice[0])

7146
['addm', 'configuration_management_database', 'servicenow', 'it_service_management', 'itil', 'service_management', 'ddm', 'bmc_remedy', 'issue_tracking', 'configuration', 'knowledge_management', 'help_desk', 'normalization', 'bppm', 'patrol', 'bmc_patrol', 'event_management', 'systems_management', 'performance_management', 'awr', 'tkprof', 'sql_trace', 'data_pump', 'oracle_dba', 'oracle_grid_control', 'statspack', 'tde', 'recovery', 'mef', 'vldb', 'nmon']


In [3]:
data_must_have = pd.read_csv('/kaggle/input/skills/mustHaveSkills-2.csv', header = 0, encoding='ISO-8859-1')
del data_must_have['job_brief_id']

print (len(data_must_have))
data_must_have = data_must_have.drop_duplicates(subset=['keyword_name', 'job_title'], keep='last')
data_must_have = data_must_have[data_must_have["job_title"] != 0]
print (len(data_must_have))
data_must_have['Count'] = data_must_have.groupby('job_title')['keyword_name'].transform(pd.Series.value_counts)
data_must_have.drop_duplicates(inplace=True)
data_must_have['keyword_name'] = data_must_have['keyword_name'].str.lower()
#data_must_have['keyword_name'] = data_must_have['keyword_name'].str.replace(' ' ,'_')
data_must_have['job_title'] = data_must_have['job_title'].str.lower()
# gr_df_keywordname = data_must_have.groupby('keyword_name')['job_title'].apply(list)
gr_df_jobtitle = data_must_have.groupby('job_title')['keyword_name'].apply(list)

gr_df_jobtitle

261724
80810


job_title
.net                             [microsoft office sharepoint server]
.net application                       [architect, .net, asp.net, c#]
.net developer      [.net, sharepoint, moss, asp.net, html/html5, ...
.net engineer       [.net, .net, asp.net, javascript, mobile appli...
.net programmer                                                [.net]
                                          ...                        
xbox security                            [c++, xbox, ps, playstation]
xd                  [ux design, ux, uxd, ued, xd, user experience ...
xml developer                                        [xml, html, css]
yield manager       [yield, online media, digital, advertising, br...
zbb analysts           [zerbased budgeting, zbb, accounting, finance]
Name: keyword_name, Length: 4596, dtype: object

In [4]:
must_have_data = []
for vector_list in gr_df_jobtitle:
    xx = list(set(vector_list))
    if xx not in must_have_data:
        must_have_data.append(xx)

len(must_have_data)
must_have_data[0:2]

[['microsoft office sharepoint server'],
 ['c#', '.net', 'architect', 'asp.net']]

In [5]:
data_naukri = pd.read_csv('/kaggle/input/skills/naukri_skill_full.txt', header = 0, encoding='ISO-8859-1')
# drop duplicate
data_naukri.drop_duplicates(subset=['id', 'skill'], keep='last')
# lower-case
data_naukri['skill'] = data_naukri['skill'].str.lower()
#data_naukri['skill'] = data_naukri['skill'].str.replace(' ','_')
data_naukri_final = data_naukri.groupby('id')['skill'].apply(list)
print(len(data_naukri_final))
data_naukri_final[:10]

120107


id
0    [ceaser, caad, design, piping, thermal power p...
1    [3d modelling, drawing, design, detail enginee...
2    [salary, international voice process, internat...
3    [international voice process, international in...
4    [m.i.s. costing, marketing support, budgeting,...
5    [.net developer, asp.net, wcf, agile methodolo...
6    [2008, wcf web services, sql server, .net, .ne...
7    [mvvm pattern, ria, linq, hibernate, soa, wpf,...
8    [rest, javascript, entity framework, mvc, jque...
9    [.net support analyst, sql queries, .net, sql,...
Name: skill, dtype: object

In [6]:
data_train_w2v = data_dice
for must_have in gr_df_jobtitle:
    if len(must_have) > 2:
        if must_have not in data_train_w2v:
            data_train_w2v.append(must_have)
            
len(data_train_w2v)

9773

In [7]:
for skills in data_naukri_final:
    if len(skills) > 2 and skills not in data_train_w2v:
        data_train_w2v.append(skills)
        
len(data_train_w2v)

107681

In [8]:
data_train_w2v_for_check = data_train_w2v
data_train_w2v = []

for i in data_train_w2v_for_check:
    vector =[]
    for j in i:
        # TODO: remove stopword, clean
        if isinstance(j, str):
            if 1 == 1: #  or j not in vector:
                vector.append(j)
    if i not in data_train_w2v:
        data_train_w2v.append(vector)

print(len(data_train_w2v))

107620


In [9]:
with open('/kaggle/working/duyet_data_train_w2v', 'wb') as f:
    pkl.dump(data_train_w2v, f)

In [10]:
import multiprocessing
from gensim.models import Word2Vec
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
print("Training model...")
model = Word2Vec(data_train_w2v,
                 workers=multiprocessing.cpu_count(),
                 vector_size=300,
                 min_count=1,
                 window=10,
                 sample=1e-3,
                 epochs=4,
                 sg=1)

Training model...


In [11]:
model.wv.save_word2vec_format('duyet_word2vec_skill.bin', binary=True)

In [12]:
model.wv.most_similar("machine learning")

[('natural language processing', 0.9320260882377625),
 ('artificial intelligence', 0.9185497760772705),
 ('data scientist', 0.9141143560409546),
 ('data science', 0.9088523387908936),
 ('nlp', 0.9018080234527588),
 ('information retrieval', 0.9013006091117859),
 ('mapreduce', 0.8965206742286682),
 ('algorithm development', 0.893947958946228),
 ('text mining', 0.8928377032279968),
 ('h2o', 0.8853837251663208)]

In [13]:
model.wv.most_similar("bda")

[('internal audit reporting', 0.9830041527748108),
 ('distribution agreements', 0.9798790216445923),
 ('reorganization', 0.9794964790344238),
 ('kyc documents', 0.9782647490501404),
 ('legal function', 0.9777449369430542),
 ('govt authorities', 0.9777047038078308),
 ('cash manager', 0.9776397347450256),
 ('mergers & acquisitions', 0.9775639772415161),
 ('design &amp; tech specification study', 0.9775562882423401),
 ('property law', 0.9773688316345215)]

In [14]:
model.wv.most_similar("business development")

[('bd', 0.8422835469245911),
 ('corporate sales', 0.8058608174324036),
 ('bdm', 0.7964255213737488),
 ('business development manager', 0.7902951836585999),
 ('new business development', 0.7792451977729797),
 ('b2b sales', 0.7737407684326172),
 ('bde', 0.7675458788871765),
 ('market research', 0.7621756792068481),
 ('business development executive', 0.7590961456298828),
 ('business generation', 0.7574049830436707)]

In [15]:
model.wv.most_similar("inside sales")

[('it sales', 0.9221687316894531),
 ('software sales', 0.915357232093811),
 ('leads', 0.8985766172409058),
 ('international sales', 0.8972474336624146),
 ('sales consulting', 0.8888075351715088),
 ('international clients', 0.8855923414230347),
 ('us sales', 0.8832508325576782),
 ('azuga', 0.8812547922134399),
 ('cold calling', 0.8799751400947571),
 ('website sales', 0.8793432116508484)]

In [16]:
model.wv.most_similar('it recruiter')

[('technical recruiter', 0.9909237623214722),
 ('recruitment consultant', 0.9713572263717651),
 ('recruitment executive', 0.9652367830276489),
 ('non it recruiter', 0.9648768305778503),
 ('senior recruiter', 0.964295506477356),
 ('dice', 0.9634674787521362),
 ('us recruitment', 0.9619625210762024),
 ('hr recruiter', 0.9615435004234314),
 ('contract to hire', 0.9613050818443298),
 ('non it recruitment', 0.9607042074203491)]

## Getting predictions on our DATASET

In [17]:
df = pd.read_excel('/kaggle/input/acceptedog-rejectedfake/acceptedOGrejectedFAKE.xlsx')

In [18]:
df.sample(5)

,status,experience,present_ctc,expected_ctc,notice_period,candidate_profiles,job_categories,designation,Relevant,exp_min,exp_max,ctc_fixed_min,ctc_fixed_max
6345,ACCEPTED,3.0,10.50,18.0,30,Associate Program Manager|||Associate Software...,Program Manager,Program Manager/Sr. Program manager,1,2.0,6.0,22.0,25.0
11508,ACCEPTED,16.0,6.00,9.0,90,Deputy Manager|||Manager|||Operations Manager,Operations,CAD Operations,1,3.0,8.0,5.0,12.0
15905,ACCEPTED,7.0,5.75,7.0,90,Account Executive|||Business Associate|||Senio...,Audit/Audit Manager,Accounts Executive,1,2.0,5.0,4.5,7.5
3142,ACCEPTED,3.5,10.80,9.0,30,Senior Software Engineer,Developer,SDE,1,1.0,2.0,6.0,8.0
450,ACCEPTED,5.0,4.20,4.8,10,GRAPHIC DESIGNER,"Designer,Graphic designer",Graphic designer,1,2.0,5.0,5.0,7.0


In [19]:
df = df.dropna()

In [20]:
import re
def clean_text_columns(df, columns):
    clean_pattern = r'[!@#$%^&*()_+{}\[\-]:;<>,.?~\\/|"]'
    for col in columns:
        df[col] = df[col].apply(lambda x: re.sub(clean_pattern, ' ', str(x)))
        df[col] = df[col].str.replace(r'\s+', ' ', regex=True).str.strip()
        df[col] = df[col].str.lower()  # Convert to lowercase
    return df

# List of columns to clean
columns_to_clean = ['candidate_profiles','designation']

# Call the clean_text_columns function to clean the specified columns
df = clean_text_columns(df, columns_to_clean)
df.sample(10)

,status,experience,present_ctc,expected_ctc,notice_period,candidate_profiles,job_categories,designation,Relevant,exp_min,exp_max,ctc_fixed_min,ctc_fixed_max
870,ACCEPTED,3.00,6.20,7.20,15,graphic designer,"Designer,Graphic designer",graphic designer,1,2.0,5.0,5.0,7.0
2717,ACCEPTED,12.00,11.05,12.71,30,dy manager|||manager pricing north east india|...,Business development,regional manager ocean freight north,1,5.0,15.0,15.0,25.0
1068,ACCEPTED,3.00,4.35,6.60,30,php developer at,Developer,full stack developer,1,2.0,4.0,4.0,4.8
7251,ACCEPTED,5.00,16.00,18.00,30,senior sales manager,Customer relationship manager,senior account executive,1,5.0,10.0,25.0,36.0
6906,ACCEPTED,4.50,5.05,6.00,7,customer support executive|||inside sales spec...,Inside Sales (Outbound),inside sales executive,1,0.6,5.0,5.0,6.0
9511,ACCEPTED,5.50,18.00,25.00,30,software engineer,Developer,se/sse ror developer,1,3.0,6.0,20.0,35.0
10628,ACCEPTED,8.00,8.20,11.00,45,deputy manager,Operations,specialist ã¢â‚¬â€œ supply planning,1,2.0,5.0,9.0,12.0
1267,ACCEPTED,3.67,2.55,2.93,15,junior executive|||trainee operating engineer,Supply chain operations (Shipping),operations executive- ocean,1,3.0,7.0,1.5,3.0
16569,REJECTED,1.20,2.75,3.80,15,supply chain manager,Creative,brand strategist,0,1.0,2.0,2.8,3.6
16167,REJECTED,6.00,10.00,15.00,30,healthcare administrator,Digital marketing,customer service representative,0,3.0,7.0,12.0,35.0


In [22]:
# Reset the index to ensure a consistent sequence of integers
df.reset_index(drop=True, inplace=True)

# Create empty lists to store predicted skills
predicted_skills_profiles = []
predicted_skills_designations = []

# Predicting skills for candidate profiles and designations
for i in range(len(df)):  # Iterate through the DataFrame indices
    profile = df['candidate_profiles'][i]
    designation = df['designation'][i]

    # Tokenize the text in candidate profiles and designations
    profile_tokens = [token.lower() for token in profile.split() if token.lower() in model.wv]
    designation_tokens = [token.lower() for token in designation.split() if token.lower() in model.wv]

    # Find most similar skills based on candidate profiles and designations
    if profile_tokens:
        predicted_skills_profiles.append([skill[0] for skill in model.wv.most_similar(positive=profile_tokens, topn=5)])
    else:
        predicted_skills_profiles.append([])

    if designation_tokens:
        predicted_skills_designations.append([skill[0] for skill in model.wv.most_similar(positive=designation_tokens, topn=5)])
    else:
        predicted_skills_designations.append([])

# Add the predicted skills to the DataFrame
df['candidate_skills'] = predicted_skills_profiles
df['jd_skills'] = predicted_skills_designations

# Reorder the columns to include predicted skills while keeping all original columns
columns_order = ['candidate_profiles', 'candidate_skills', 'designation', 'jd_skills'] + [col for col in df.columns if col not in ['candidate_profiles', 'designation']]
df = df[columns_order]

# Export the updated DataFrame to a CSV file
df.to_csv('predicted_skills_combined_with_original_columns.csv', index=False)